Notebook includng some peripheral stuff. This includes the gmaps API and pulling price estamates from Lyft and Uber with Selenium.

As a side note, the Lyft and Uber API's are basically useless as they require being a 'partner' to use.

### Dynamic web scraping
- Nice tutorial here: https://realpython.com/modern-web-automation-with-python-and-selenium/
- and the selenium docs: https://selenium-python.readthedocs.io/locating-elements.html

In [108]:
import selenium
from selenium.webdriver.common.keys import Keys
import time

In [90]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.headless = False
#assert opts.headless  # Operating in headless mode

In [91]:
# Simple scrape of duckduckgo
browser = Firefox(options=opts)
browser.get('https://duckduckgo.com')

search_form = browser.find_element_by_name('q')
search_form.send_keys('real python')
search_form.submit()

In [16]:
results = browser.find_elements_by_class_name('result')
print(results[0].text)
browser.close()

Python Tutorials - Real Python
https://realpython.com
At Real Python you can learn all things Python from the ground up. If you're wondering where to begin your Python journey, click the button below and we'll give you some guidance: Start Your Python Journey »


In [183]:
ESTIMATE_URLS = {"lyft": "https://www.lyft.com/rider/fare-estimate",
                 "uber": "https://www.uber.com/us/en/price-estimate"}
browser = Firefox(options=opts)
browser.get(ESTIMATE_URLS['lyft'])

In [184]:
start_field = browser.find_element_by_name('fare-start')
end_field = browser.find_element_by_name('fare-end')

start_field.send_keys('Milennium Park chicago il')
time.sleep(2)
start_field.send_keys(Keys.RETURN)
end_field.send_keys('Willis Tower chicago il')
time.sleep(2)
end_field.send_keys(Keys.RETURN)
time.sleep(5)
tags = browser.find_elements_by_tag_name("td")
browser.close()

#button = browser.find_element_by_css_selector("button[type='submit']")
#button.click()

tag
#end_field.submit()

In [191]:
def lyft_table(tags):
    name_offset = 1
    value_offset = 2
    cols = 4
    rows = int(len(tags)/cols)
    results = {}
        
    for j in range(rows):
        # extract the data
        name = tags[cols*j+name_offset].text
        values = tags[cols*j+value_offset].text
        
        # parse the string
        values = values.replace('$','').replace('min','')
        prices, time = values.split('|')
        price_l, price_h = prices.split('-')
        
        #build results
        results[name] = {'price_l':price_l,'price_h':price_h,'time_min':time}


    return results

In [198]:
def lyft_estimate(start,stop,browser):
    browser.get(ESTIMATE_URLS['lyft'])
    
    start_field = browser.find_element_by_name('fare-start')
    end_field = browser.find_element_by_name('fare-end')

    # TODO: Replace time.sleep with wait for popup on website
    # TODO: make sure the start and stop don't become something cray
    start_field.send_keys(start)
    time.sleep(2)
    start_field.send_keys(Keys.RETURN)
    end_field.send_keys(stop)
    time.sleep(2)
    end_field.send_keys(Keys.RETURN)
    tags = browser.find_elements_by_tag_name("td")
    return lyft_table(tags)

In [200]:
ESTIMATE_URLS = {"lyft": "https://www.lyft.com/rider/fare-estimate",
                 "uber": "https://www.uber.com/us/en/price-estimate"}
opts = Options()
opts.headless = True
browser = Firefox(options=opts)
browser.implicitly_wait(5) # time to allow objects to load on page

lyft_prices = lyft_estimate('brooklyn', 'empire state building', browser)

browser.close()

In [197]:
lyft_prices

{'Personal ride': {'price_l': '65', 'price_h': '70 ', 'time_min': ' 57'},
 'Shared ride': {'price_l': '55', 'price_h': '60 ', 'time_min': ' 57'},
 'Supersized ride': {'price_l': '70', 'price_h': '77 ', 'time_min': ' 57'},
 'Luxury ride': {'price_l': '100', 'price_h': '110 ', 'time_min': ' 57'},
 'Spacious luxury ride': {'price_l': '142',
  'price_h': '154 ',
  'time_min': ' 57'}}

### Google Maps API

- Geocoding:
- https://geocoder.readthedocs.io/providers/Mapbox.html

Looks like it will be simple to translate user text to lat/long and get google routing estimates!
Way easier than lyft and uber's API.

In [202]:
import googlemaps
from datetime import datetime
import json

with open("secrets.json.nogit") as fh:
    secrets = json.loads(fh.read())

gmaps = googlemaps.Client(key=secrets['gmaps_api_key'])

# Geocoding an address
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions("Sydney Town Hall",
                                    "Parramatta, NSW",
                                    departure_time=now)

In [235]:
## note that the directions_result gives basically gives the geocode information but finds the nearest street
## which is what we want

def geocoded_path(start,stop):
    start_geo = gmaps.geocode(start)
    end_geo = gmaps.geocode(stop)
    
    now = datetime.now()
    directions_result = gmaps.directions(start,
                                    stop,
                                    departure_time=now)
    
    start_coords = tuple(start_geo[0]['geometry']['location'].values())
    end_coords = tuple(end_geo[0]['geometry']['location'].values())
    
    return start_coords, end_coords, start_geo, end_geo, directions_result

In [241]:
a, b, c, d, e = geocoded_path("willis tower","millenium park")

In [315]:
e

[{'bounds': {'northeast': {'lat': 41.8808498, 'lng': -87.6227524},
   'southwest': {'lat': 41.8787965, 'lng': -87.6352812}},
  'copyrights': 'Map data ©2019 Google',
  'legs': [{'distance': {'text': '0.8 mi', 'value': 1241},
    'duration': {'text': '6 mins', 'value': 343},
    'duration_in_traffic': {'text': '7 mins', 'value': 391},
    'end_address': 'Millennium Park, 201 E Randolph St, Chicago, IL 60602, USA',
    'end_location': {'lat': 41.8808498, 'lng': -87.6227524},
    'start_address': '233 S Wacker Dr, Chicago, IL 60606, USA',
    'start_location': {'lat': 41.8787965, 'lng': -87.63525010000001},
    'steps': [{'distance': {'text': '0.1 mi', 'value': 203},
      'duration': {'text': '1 min', 'value': 57},
      'end_location': {'lat': 41.8806221, 'lng': -87.6352812},
      'html_instructions': 'Head <b>north</b> on <b>S Franklin St</b> toward <b>W Adams St</b>',
      'polyline': {'points': 'omr~Fhg{uOA?gA@c@?E?gA@eA?a@@gA?'},
      'start_location': {'lat': 41.8787965, 'lng': 

In [267]:
e[0]['legs'][0]['duration_in_traffic']
#['duration_in_traffic']

{'text': '7 mins', 'value': 391}

In [316]:
GMAPS_NAME_MAPPING = {'start_address':'start_address',
                      'end_address':'end_address',
                      'start_location':'start_latlong',
                      'end_location':'end_latlong',
                      'duration_in_traffic':'time_s',
                      'distance':'dist_m'}

def parse_gmaps(directions_result):
    dr = directions_result[0]['legs'][0]
    out = {}
    for gname,newname in GMAPS_NAME_MAPPING.items():
        vals = dr[gname]
        if gname == 'start_address' or gname == 'end_address':
            out[newname] = vals
        elif gname == 'end_location' or gname =='start_locaton':
            print(vals)
            out[newname] = tuple([float(val) for val in vals.values()])
        else:
            out[newname] = int(list(vals.values())[1])
    return out
        

In [317]:
parse_gmaps(e)
#dr['start_address']

{'lat': 41.8808498, 'lng': -87.6227524}


{'start_address': '233 S Wacker Dr, Chicago, IL 60606, USA',
 'end_address': 'Millennium Park, 201 E Randolph St, Chicago, IL 60602, USA',
 'start_latlong': -87,
 'end_latlong': (41.8808498, -87.6227524),
 'time_s': 391,
 'dist_m': 1241}